In [3]:
import vesuvius
from vesuvius import Volume
from matplotlib import pyplot as plt
import numpy as np
segment_id = 20231007101619
segment = Volume(segment_id, normalize=True)
print(f"volume loaded with shape {segment.shape}")
print(segment[32, 5000:5005, 5000:5005])
def count_zeros_ones_iter(arr):
    zeros = ones = 0
    it = np.nditer(arr, flags=['refs_ok', 'zerosize_ok'])
    for val in it:
        if val == 0:
            zeros += 1
        elif val == 1:
            ones += 1
    return zeros, ones

zeros, ones = count_zeros_ones_iter(segment.inklabel)
print(f"Zeros: {zeros}, Ones: {ones}")

Failed to download inklabel. Status code: 404
volume loaded with shape <bound method Volume.shape of <vesuvius.volume.Volume object at 0x7ac7d00b58a0>>
[[0.5372549  0.48627451 0.4        0.31764706 0.25882353]
 [0.45882353 0.42745098 0.38039216 0.33333333 0.29411765]
 [0.49411765 0.47843137 0.4627451  0.44313725 0.41960784]
 [0.63137255 0.60392157 0.56862745 0.54117647 0.50196078]
 [0.72941176 0.6627451  0.57647059 0.50588235 0.44313725]]


KeyboardInterrupt: 

In [ ]:
import vesuvius
from vesuvius import Volume
from matplotlib import pyplot as plt
import numpy as np
segment_id = 20231007101619
segment = Volume(segment_id, normalize=True)

def count_zeros_ones(arr):
    zeros = np.count_nonzero(arr == 0)
    ones = np.count_nonzero(arr == 1)
    return zeros, ones
zeros, ones = count_zeros_ones(segment.inklabel)
print(f"Zeros: {zeros}, Ones: {ones}")

Failed to download inklabel. Status code: 404
